## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [3]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = []
col_eliminar_tarjetas = ['nivel_tarjeta']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [4]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,24.0,2.0


##### Del análisis realizados se definieron las siguientes características a ser eliminadas de dataset datos_creditos.csv:

*antiguedad_empleado: se elimina este atributo ya que no aporta valor en el análisis, además que tiene un 3.35% de filas con valores fuera de rango.

*tasa_interes: se elimina esta columna ya que en el análisis de calidad de los datos se evidenció que este atributo tiene el 9.01% de filas con valores fuera de rango.


##### Del análisis realizados se definieron las siguientes características a ser eliminadas de dataset datos_tarjetas.csv:

*genero: no se considera un aspecto importante para la empresa.

*limite_credito_tc: no es un factor determinante para aprobar o no un crédito.

*nivel_educativo: se elimina este atributo dato que el 15% de los registros tienen un dato "Desconocido", por lo cual no se podría considerar para el análisis.

*personas_a_cargo: el numero de personas a cargo no genera algún aporte en el análisis.



In [5]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['antiguedad_empleado', 'tasa_interes']
col_eliminar_tarjetas = ['genero', 'limite_credito_tc', 'nivel_educativo','personas_a_cargo']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [6]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,operaciones_ult_12m
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,24.0


Limpieza de los datos (filtrado a nivel de filas)

In [7]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,1,Y,ok
1,768805383.0,21,1000,2,PROPIA,9600,EDUCACIÓN,0.10,0,N,ok
2,818770008.0,25,5500,3,HIPOTECA,9600,SALUD,0.57,1,N,err
3,713982108.0,23,35000,2,ALQUILER,65500,SALUD,0.53,1,N,ok
4,710821833.0,24,35000,4,ALQUILER,54400,SALUD,0.55,1,Y,ok


In [8]:
#Aplicación de la 2da regla
# Identificar la duración mínima de los créditos
duracion_minima = df_creditos['duracion_credito'].min()

# Definir la función para aplicar la regla
def regla_duracion_minima(row):
    if row['duracion_credito'] == duracion_minima:
        if row['pct_ingreso'] > 0.6 and row['situacion_vivienda'] != 'PROPIA':
            return 'err'
    return 'ok'

# Aplicar la función para todos los elementos del dataset
df_creditos['regla_duracion_minima'] = df_creditos.apply(regla_duracion_minima, axis=1)

# Visualizar las filas que no cumplen con la regla
print("Se visualizan las tuplas que no cumplen con la regla de duración mínima:\n")
display(df_creditos[df_creditos['regla_duracion_minima'] == 'err'])

# Contar las filas que no cumplen la regla
cantidad_erroneas = df_creditos[df_creditos['regla_duracion_minima'] == 'err'].shape[0]
print(f"Cantidad de filas que no cumplen la regla de duración mínima: {cantidad_erroneas}")

Se visualizan las tuplas que no cumplen con la regla de duración mínima:



,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla_pct_ingresos,regla_duracion_minima
36,713962233.0,22,30000,2,ALQUILER,48000,EDUCACIÓN,0.63,1,N,ok,err
571,709040508.0,21,15000,2,HIPOTECA,19500,EDUCACIÓN,0.77,1,N,err,err
577,716027208.0,26,20050,2,ALQUILER,26000,EDUCACIÓN,0.77,1,N,ok,err
587,709186983.0,23,20000,2,ALQUILER,32900,EDUCACIÓN,0.61,1,N,ok,err
3535,717886008.0,24,25000,2,HIPOTECA,37000,SALUD,0.68,0,N,ok,err
6758,713115483.0,22,7000,2,ALQUILER,10000,MEJORAS_HOGAR,0.70,1,N,err,err
7220,711399408.0,24,29100,2,HIPOTECA,46000,PERSONAL,0.63,1,Y,ok,err


Cantidad de filas que no cumplen la regla de duración mínima: 7


In [10]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[temp['regla_pct_ingresos'] == 'ok']

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10120
Filas después del filtro: 10103


In [11]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

# Eliminar las filas que no cumplen con la regla
df_creditos = df_creditos[df_creditos['regla_duracion_minima'] == 'ok']


print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10120
Filas después del filtro: 10103


Integración de datos

In [12]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 10103


In [13]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 17


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [14]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

df_final = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_final.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,estado_credito_N,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla_pct_ingresos,regla_duracion_minima,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,operaciones_ult_12m
0,C,C,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,1,Y,ok,ok,36.0,CASADO,ACTIVO,1088.0,24.0
1,C,P,768805383.0,21,1000,2,PROPIA,9600,EDUCACIÓN,0.10,0,N,ok,ok,39.0,CASADO,ACTIVO,1144.0,42.0
2,C,C,713982108.0,23,35000,2,ALQUILER,65500,SALUD,0.53,1,N,ok,ok,36.0,CASADO,ACTIVO,1887.0,20.0
3,C,C,710821833.0,24,35000,4,ALQUILER,54400,SALUD,0.55,1,Y,ok,ok,54.0,CASADO,ACTIVO,1314.0,26.0
4,N,C,769911858.0,21,2500,2,PROPIA,9900,INVERSIONES,0.25,1,N,ok,ok,34.0,DESCONOCIDO,ACTIVO,1171.0,20.0


Atributos numéricos que se discretizan

In [15]:
df_integrado['operaciones_ult_12m'].describe()

count    10103.000000
mean        64.901019
std         23.466474
min         10.000000
25%         45.000000
50%         67.000000
75%         81.000000
max        139.000000
Name: operaciones_ult_12m, dtype: float64

In [16]:
# Antiguedad del empleado

#etiquetas_a_e = ['menor_10', '5_a_10', 'mayor_10']
#rangos_a_e = [0, 4, 10, 50]
#valor_para_nan = 'NA'
#antiguedad_empleados_N = pd.cut(df_integrado['antiguedad_empleado'], 
#                                bins=rangos_a_e, 
#                                labels=etiquetas_a_e,
#                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

#antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

#etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
#rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
#tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
#                                bins=rangos_t_i, 
#                                labels=etiquetas_t_i)

#tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# limite_credito_tc

#etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
#rangos_l_tc = [0, 2999, 4999, 9999, 100000]
#limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
#                                bins=rangos_l_tc, 
#                                labels=etiquetas_l_tc)

#limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50_a_75      3391
75_a_100     3004
30_a_50      2410
mayor_100     687
15_a_30       587
menor_15       24
Name: count, dtype: int64

In [17]:
df_integrado.columns

Index(['id_cliente', 'edad', 'importe_solicitado', 'duracion_credito',
       'situacion_vivienda', 'ingresos', 'objetivo_credito', 'pct_ingreso',
       'estado_credito', 'falta_pago', 'regla_pct_ingresos',
       'regla_duracion_minima', 'antiguedad_cliente', 'estado_civil',
       'estado_cliente', 'gastos_ult_12m', 'operaciones_ult_12m'],
      dtype='object')

In [18]:
col_eliminar_final = [
              'edad',
              #'antiguedad_empleado',
              'antiguedad_cliente', 
              'ingresos',
              'pct_ingreso', 
              #'tasa_interes',
              'regla_pct_ingresos',
              'gastos_ult_12m', 
              #'limite_credito_tc', 
              'operaciones_ult_12m',
              'id_cliente']

df_integrado.drop(col_eliminar_final, inplace=True, axis=1)


df_final = pd.concat([operaciones_ult_12m_N, gastos_ult_12m_N, antiguedad_cliente_N, ingresos_N, pct_ingreso_N, edad_N, df_integrado], axis=1)
df_final.head(5)


,operaciones_ult_12m,gastos_ult_12m,antiguedad_cliente,ingresos,pct_ingreso,edad,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,estado_credito,falta_pago,regla_duracion_minima,estado_civil,estado_cliente
0,15_a_30,2k_a_4k,2y_a_4y,50k_a_100k,40_a_60,menor_25,35000,3,ALQUILER,PERSONAL,1,Y,ok,CASADO,ACTIVO
1,30_a_50,2k_a_4k,2y_a_4y,hasta_20k,hasta_20,menor_25,1000,2,PROPIA,EDUCACIÓN,0,N,ok,CASADO,ACTIVO
2,15_a_30,2k_a_4k,2y_a_4y,50k_a_100k,40_a_60,menor_25,35000,2,ALQUILER,SALUD,1,N,ok,CASADO,ACTIVO
3,15_a_30,2k_a_4k,mayor_4y,50k_a_100k,40_a_60,menor_25,35000,4,ALQUILER,SALUD,1,Y,ok,CASADO,ACTIVO
4,15_a_30,2k_a_4k,2y_a_4y,hasta_20k,20_a_40,menor_25,2500,2,PROPIA,INVERSIONES,1,N,ok,DESCONOCIDO,ACTIVO


In [19]:
col_eliminar_final_2 = [
              'estado_civil',
              'estado_credito']

df_final.drop(col_eliminar_final_2, inplace=True, axis=1)

df_final = pd.concat([estado_civil_N, estado_credito_N, df_final], axis=1)
df_final.head()

,estado_civil_N,estado_credito_N,operaciones_ult_12m,gastos_ult_12m,antiguedad_cliente,ingresos,pct_ingreso,edad,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,falta_pago,regla_duracion_minima,estado_cliente
0,C,C,15_a_30,2k_a_4k,2y_a_4y,50k_a_100k,40_a_60,menor_25,35000,3,ALQUILER,PERSONAL,Y,ok,ACTIVO
1,C,P,30_a_50,2k_a_4k,2y_a_4y,hasta_20k,hasta_20,menor_25,1000,2,PROPIA,EDUCACIÓN,N,ok,ACTIVO
2,C,C,15_a_30,2k_a_4k,2y_a_4y,50k_a_100k,40_a_60,menor_25,35000,2,ALQUILER,SALUD,N,ok,ACTIVO
3,C,C,15_a_30,2k_a_4k,mayor_4y,50k_a_100k,40_a_60,menor_25,35000,4,ALQUILER,SALUD,Y,ok,ACTIVO
4,N,C,15_a_30,2k_a_4k,2y_a_4y,hasta_20k,20_a_40,menor_25,2500,2,PROPIA,INVERSIONES,N,ok,ACTIVO


In [20]:
df_final.shape[1]

15

Exportación de metadatos

In [23]:
import dtale as dt

informe = dt.show(df_final)
informe.open_browser()

d:\Gestion_proyectos\Practica1\13MBID-OCT2324\.venv\Lib\site-packages\dtale\column_analysis.py:402: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

d:\Gestion_proyectos\Practica1\13MBID-OCT2324\.venv\Lib\site-packages\dtale\column_analysis.py:402: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

d:\Gestion_proyectos\Practica1\13MBID-OCT2324\.venv\Lib\site-packages\dtale\column_analysis.py:402: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

d:\Gestion_proyectos\Practica1\13MBID-OCT2324\.venv\Lib\site-packages\dtale\column_analysis.py:402: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

d:\Gestion_proyectos\Practica1\13MBID-OCT2324\.venv\Lib\site

Exportación del dataset

In [22]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)